# Exercise 3

Tianfang Wang  a1788845

## Loading modules

In [40]:
import os
import re
import numpy as np
# Specify git executable file for GitPython in Jupyter Notebook (In IDE, it can still work without this line.)
os.environ["GIT_PYTHON_REFRESH"] = "quiet"
import git
from git import RemoteProgress
from git import Repo


## Find the smallest enclosing scope

In [41]:
def sumLines(dLines):
    pattern = re.compile('@@')
    return [s for s in dLines if pattern.match(s)]

def pSummary(summary):
    num = summary.split(' ')
    dNum = num[1].split(',')
    aNum = num[2].split(',')
    dStart = -int(dNum[0])
    if len(dNum)==1:
        dLength = 1
    else:
        dLength = int(dNum[1])
    aStart = int(aNum[0])
    if len(aNum)==1:
        aLength = 1
    else:
        aLength = int(aNum[1])
    return (dStart, dLength, aStart, aLength)

def findScope(dStart, dLength, content):
    lineNum = dStart-1
    count = 1
    while count > 0:
        lineNum += 1
        if lineNum >= len(content):
            lineNum = len(content) - 1
            break
        line = content[lineNum]
        for c in line:
            if c=='}':
                count -= 1
                if count==0:
                    break
            if c=='{':
                count += 1
    sEnd = lineNum+1
    lineNum = dStart
    count = 1
    while count > 0:
        lineNum -= 1
        if lineNum < 0:
            lineNum = 0
            break
        line = content[lineNum]
        for c in line[::-1]:
            if c=='{':
                count -= 1
                if count==0:
                    break
            if c=='}':
                count += 1
    sStart = lineNum+1
    return (sStart, sEnd)

## Identify the lastest commit

In [42]:
def lastestCommit(blames):
    commits = []
    times = []
    for b in blames:
        tmp = b.split()
        commits += [tmp[0]]
        times += [int(tmp[3])]
    return commits[np.argmax(times)]

## Find VCC

In [43]:
def findVCC(repo_path, fixing_commit):
    repo = Repo(repo_path)
    HEAD = fixing_commit
    PREV = fixing_commit + '^'
    files = repo.git.diff('--name-only', PREV, HEAD).splitlines()
    commitsFound = []
    
    for file in files:
        try:
            content = repo.git.show(PREV+':'+file).splitlines()
        except:
            continue
        dLines = repo.git.diff('-U0',PREV,HEAD,file).splitlines()
        summlines = sumLines(dLines)
        for line in summlines:
            (dStart, dLength, aStart, aLength) = pSummary(line)
            if dLength > 0:
                blames = repo.git.blame('-w','--date=unix','-e','-f','-L '+str(dStart) + ',' + str(dStart + dLength - 1), PREV, file).splitlines()
                commit = lastestCommit(blames)
                commitsFound += [commit]*dLength
            if aLength > 0:
                (sStart, sEnd) = findScope(dStart, dLength, aStart, aLength, content)
                blames = repo.git.blame('-w','--date=unix','-e','-f','-L '+str(sStart) + ',' + str(sEnd), PREV, file).splitlines()
                commit = lastestCommit(blames)
                commitsFound += [commit]*aLength
    VCC = max(set(commitsFound), key=commitsFound.count)
    return VCC

## Case 1 CVE-2015-5174

In [44]:
remote_link = "https://github.com/apache/tomcat80"
local_link = "~/Documents/tomcat80"
if not os.path.isdir(local_link):
    Repo.clone_from(remote_link, local_link, progress = Progress())
fixing_commit = "2fc9d03ffbc3fe7eabfd272380807ac0ddcf748d"
vcc = find_vcc(local_link, fixing_commit)
print("\nCase 1 repository: %s, commit: %s" % (remote_link, fixing_commit))
print("\nVCC is ", vcc)


Case 1 repository: https://github.com/apache/tomcat80, commit: 2fc9d03ffbc3fe7eabfd272380807ac0ddcf748d

VCC is  d0e4c578f6


## Case 2 CVE-2018-1325

In [45]:
remote_link = "https://github.com/sebfz1/wicket-jquery-ui"
local_link = "~/Documents/wicket-jquery-ui"
if not os.path.isdir(local_link):
    Repo.clone_from(remote_link, local_link, progress = Progress())
fixing_commit = "2fec03dbe2f6e8808f4bdc6b3195dff3e44f520"
vcc = find_vcc(local_link, fixing_commit)
print("\nCase 2 repository: %s, commit: %s" % (remote_link, fixing_commit))
print("\nVCC is ", vcc)


Case 2 repository: https://github.com/sebfz1/wicket-jquery-ui, commit: 2fec03dbe2f6e8808f4bdc6b3195dff3e44f520

VCC is  a7b9f7961e


## Case 3 CVE-2013-7251

In [46]:
remote_link = "https://github.com/micromata/projectforge-webapp"
local_link = "~/Documents/projectforge-webapp"
if not os.path.isdir(local_link):
    Repo.clone_from(remote_link, local_link, progress = Progress())
fixing_commit= "422de35e3c3141e418a73bfb39b430d5fd74077e"
vcc = find_vcc(local_link, fixing_commit)
print("\nCase 3 repository: %s, commit: %s" % (remote_link, fixing_commit))
print("\nVCC is ", vcc)


Case 3 repository: https://github.com/micromata/projectforge-webapp, commit: 422de35e3c3141e418a73bfb39b430d5fd74077e

VCC is  96ca7e5ac0
